In [1]:
import numpy as np 
import pandas as pd 
import math 
from sklearn import preprocessing
import gmplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Masking, Input, Dropout
from keras.layers.merge import concatenate
from keras.utils import plot_model
pd.options.display.max_rows = 100

ModuleNotFoundError: ignored

In [0]:
dfs={}
for i in range(19):
	year=str(2000+i)
	file='datos/DesmatamentoMunicipios' + year + '.csv'
	dfs[i] = pd.read_csv(file, encoding = "ISO-8859-1", index_col=0, sep=",")
	#eliminamos columnas irrelevantes
	dfs[i].drop(columns='Latgms', inplace=True)
	dfs[i].drop(columns='Longms', inplace=True)
	dfs[i].drop(columns='CodIbge', inplace=True)
	#df.drop(columns='Soma', inplace=True)
	#df.rename(columns={0:'Latitud', 1:'Longitud', 2:'Municipio', 3:'Estado', 4: 'AreaKm2', 5:'Deforestacion', 6:'Incremento deforestacion', 7:'Bosque', 8:'Nubes', 9:'No observado', 10:'No bosque', 11:'Hidrografia', 12:'Check'} ,inplace=True)
	dfs[i].columns=['Latitud', 'Longitud', 'Municipio', 'Estado', 'Area total', 'Deforestacion ' + year, 'Incremento deforestacion ' + year, 'Bosque ' + year, 'Nubes ' + year, 'No observado ' + year, 'No bosque', 'Hidrografia', 'Check ' + year]
	dfs[i]=dfs[i][['Latitud', 'Longitud', 'Municipio', 'Estado', 'Area total', 'No bosque', 'Hidrografia', 'Deforestacion ' + year, 'Incremento deforestacion ' + year, 'Bosque ' + year, 'Nubes ' + year, 'No observado ' + year, 'Check ' + year]]
    


In [0]:
df=dfs[0]
dic={}
for idx, row in df.iterrows():
		if row['Municipio'] in dic:
			dic[row['Municipio']]+=1
		else:
			dic[row['Municipio']]=1

repetidos=[]
for key, value in dic.items():
	if value==2:
		repetidos.append(key)

rep= df.Municipio.isin(repetidos)
mismo_municipio =  df[rep]
print("Hay algunos municipios con el mismo nombre en 2 estados distintos:")
print(mismo_municipio.loc[:,'Latitud':'Area total'])

df=dfs[0]
for idx, value in dfs.items():   
	value['Municipio']=value['Municipio'] + " (" + value['Estado'] + ")"
	if idx > 0:
		value.drop(columns='Latitud', inplace=True)
		value.drop(columns='Longitud', inplace=True)
		value.drop(columns='Estado', inplace=True)
		value.drop(columns='Area total', inplace=True)
		value.drop(columns='Hidrografia', inplace=True)
		value.drop(columns='No bosque', inplace=True)
		df=pd.merge(df, value, on='Municipio')

pd.set_option('display.max_columns', None)

# reorder columns
cols = df.columns.tolist()
cols = cols[2:4] + cols[0:2] + cols[4:]
df = df[cols]

print("Por eso, hacemos merge por la dupla municipio-estado:")
print(df.head())

Hay algunos municipios con el mismo nombre en 2 estados distintos:
      Latitud  Longitud               Municipio Estado  Area total
Nr                                                                
69   -9.99527 -68.15925              Rio Branco     AC        9113
104 -11.19374 -61.84743       Presidente Médici     RO        1773
105  -5.02798 -48.77112  Bom Jesus do Tocantins     PA        2827
170  -7.55219 -48.99658              Pau D'Arco     TO        1381
243  -7.72486 -50.13413              Pau D'Arco     PA        1674
264  -6.73568 -48.55630                Araguanã     TO         838
345 -15.28141 -58.15176              Rio Branco     MT         529
436  -2.30799 -45.76410       Presidente Médici     MA         441
626  -9.00182 -47.86311  Bom Jesus do Tocantins     TO        1338
636  -3.06101 -45.77225                Araguanã     MA         811
Por eso, hacemos merge por la dupla municipio-estado:
                 Municipio Estado   Latitud  Longitud  Area total  No bosqu

In [0]:
file='datos/terrabrasilis_amazon.csv'
df_new = pd.read_csv(file, encoding = "UTF-8", sep=",")
#limpiamos el dataset
df_new.drop(['geocode_ibge'], axis=1, inplace=True)
df_new = df_new.groupby(['municipality', 'state', 'year'])['areakm'].agg('sum').reset_index()

In [0]:
# Observamos los estados en los 2 formatos
estados=[]
for s in set(df_new['state'].values):
    estados.append(s)
print(estados)
print(set(df['Estado'].values))

['ACRE', 'PARÁ', 'RONDÔNIA', 'RORAIMA', 'MATO GROSSO', 'MARANHÃO', 'TOCANTINS', 'AMAPÁ', 'AMAZONAS']
{'PA', 'AC', 'MT', 'RO', 'MA', 'RR', 'TO', 'AM', 'AP'}


In [0]:
df_new

,municipality,state,year,areakm
0,ABAETETUBA,PARÁ,2008,1.240613
1,ABAETETUBA,PARÁ,2009,2.987934
2,ABAETETUBA,PARÁ,2010,3.814274
3,ABAETETUBA,PARÁ,2011,0.823062
4,ABAETETUBA,PARÁ,2012,2.230644
...,...,...,...,...
6465,ÓBIDOS,PARÁ,2015,13.210723
6466,ÓBIDOS,PARÁ,2016,10.077703
6467,ÓBIDOS,PARÁ,2017,17.931261
6468,ÓBIDOS,PARÁ,2018,24.062445


In [0]:
# Definimos un diccionario para traducir los estados al mismo formato
estados = {
    'MATO GROSSO': 'MT', 
    'MARANHÃO': 'MA',
    'AMAPÁ': 'AP',
    'RORAIMA': 'RR',
    'AMAZONAS': 'AM',
    'PARÁ': 'PA',
    'RONDÔNIA': 'RO',
    'TOCANTINS': 'TO',
    "ACRE": "AC"
}

In [0]:
for index, row in df_new.iterrows():
    df_new.iloc[index,1] = estados[row['state']]
    df_new.iloc[index,0] = row['municipality'] + " (" + str(df_new.iloc[index,1]) + ")"
df_new

,municipality,state,year,areakm
0,ABAETETUBA (PA),PA,2008,1.240613
1,ABAETETUBA (PA),PA,2009,2.987934
2,ABAETETUBA (PA),PA,2010,3.814274
3,ABAETETUBA (PA),PA,2011,0.823062
4,ABAETETUBA (PA),PA,2012,2.230644
...,...,...,...,...
6465,ÓBIDOS (PA),PA,2015,13.210723
6466,ÓBIDOS (PA),PA,2016,10.077703
6467,ÓBIDOS (PA),PA,2017,17.931261
6468,ÓBIDOS (PA),PA,2018,24.062445


In [0]:
df_new.loc[df_new['municipality']=='PARAGOMINAS (PA)']

,municipality,state,year,areakm
4085,PARAGOMINAS (PA),PA,2008,63.339999
4086,PARAGOMINAS (PA),PA,2009,120.407843
4087,PARAGOMINAS (PA),PA,2010,67.920908
4088,PARAGOMINAS (PA),PA,2011,35.098496
4089,PARAGOMINAS (PA),PA,2012,17.080393
4090,PARAGOMINAS (PA),PA,2013,30.552451
4091,PARAGOMINAS (PA),PA,2014,19.701243
4092,PARAGOMINAS (PA),PA,2015,23.591894
4093,PARAGOMINAS (PA),PA,2016,19.856212
4094,PARAGOMINAS (PA),PA,2017,19.358134


In [0]:
# Upper del municipio para que tengan el mismo formato
for index, row in df.iterrows():
    df.iloc[index,0] = df.iloc[index,0].upper()
df

,Municipio,Estado,Latitud,Longitud,Area total,No bosque,Hidrografia,Deforestacion 2000,Incremento deforestacion 2000,Bosque 2000,Nubes 2000,No observado 2000,Check 2000,Deforestacion 2001,Incremento deforestacion 2001,Bosque 2001,Nubes 2001,No observado 2001,Check 2001,Deforestacion 2002,Incremento deforestacion 2002,Bosque 2002,Nubes 2002,No observado 2002,Check 2002,Deforestacion 2003,Incremento deforestacion 2003,Bosque 2003,Nubes 2003,No observado 2003,Check 2003,Deforestacion 2004,Incremento deforestacion 2004,Bosque 2004,Nubes 2004,No observado 2004,Check 2004,Deforestacion 2005,Incremento deforestacion 2005,Bosque 2005,Nubes 2005,No observado 2005,Check 2005,Deforestacion 2006,Incremento deforestacion 2006,Bosque 2006,Nubes 2006,No observado 2006,Check 2006,Deforestacion 2007,Incremento deforestacion 2007,Bosque 2007,Nubes 2007,No observado 2007,Check 2007,Deforestacion 2008,Incremento deforestacion 2008,Bosque 2008,Nubes 2008,No observado 2008,Check 2008,Deforestacion 2009,Incremento deforestacion 2009,Bosque 2009,Nubes 2009,No observado 2009,Check 2009,Deforestacion 2010,Incremento deforestacion 2010,Bosque 2010,Nubes 2010,No observado 2010,Check 2010,Deforestacion 2011,Incremento deforestacion 2011,Bosque 2011,Nubes 2011,No observado 2011,Check 2011,Deforestacion 2012,Incremento deforestacion 2012,Bosque 2012,Nubes 2012,No observado 2012,Check 2012,Deforestacion 2013,Incremento deforestacion 2013,Bosque 2013,Nubes 2013,No observado 2013,Check 2013,Deforestacion 2014,Incremento deforestacion 2014,Bosque 2014,Nubes 2014,No observado 2014,Check 2014,Deforestacion 2015,Incremento deforestacion 2015,Bosque 2015,Nubes 2015,No observado 2015,Check 2015,Deforestacion 2016,Incremento deforestacion 2016,Bosque 2016,Nubes 2016,No observado 2016,Check 2016,Deforestacion 2017,Incremento deforestacion 2017,Bosque 2017,Nubes 2017,No observado 2017,Check 2017,Deforestacion 2018,Incremento deforestacion 2018,Bosque 2018,Nubes 2018,No observado 2018,Check 2018
0,PARAGOMINAS (PA),PA,-3.12356,-47.40354,19465,7.1,36.0,7212.3,NaN,12208.2,0.0,1.4,100,7322.3,110.0,12098.2,0.0,1.4,100,7473.1,150.8,11947.0,0.4,1.4,100,7567.3,94.2,11853.2,0.0,1.4,100,7832.6,265.3,11587.9,0.0,1.4,100,8166.1,333.5,11253.9,0.5,1.4,100,8257.6,91.5,8732.4,2430.5,1.4,100,8349.1,91.5,10645.2,422.6,5.0,100,8415.4,66.3,9094.9,1904.5,7.1,99,8536.7,121.3,10703.6,121.8,59.8,100,8605.0,68.3,10036.8,776.0,4.1,100,8641.0,36.0,10301.9,474.8,4.2,100,8658.7,17.7,7272.8,3486.4,4.0,99,8689.5,30.8,7623.6,3105.0,3.8,100,8709.5,20.0,7531.4,3179.6,1.4,99,8733.7,24.2,9587.8,1100.4,0.0,100,8753.5,19.8,9640.6,1027.8,0.0,100,8773.3,19.8,10630.7,17.9,0.0,99,8791.7,18.4,5999.4,4630.8,0.0,100
1,SÃO FÉLIX DO XINGU (PA),PA,-7.46114,-52.46225,84253,3788.0,939.0,7037.4,NaN,72482.1,0.0,6.5,100,8727.7,1690.4,70791.8,0.0,6.5,100,9993.2,1265.5,69526.3,0.0,6.5,100,11311.3,1318.0,68208.2,0.0,6.5,100,12393.8,1082.5,67125.7,0.0,6.5,100,13801.7,1407.8,65717.8,0.0,6.5,100,14563.6,761.9,64955.7,0.2,6.5,100,15441.1,877.5,63083.3,995.0,6.6,100,16206.2,765.1,63179.6,133.7,6.5,100,16650.6,444.4,60981.6,1885.2,8.6,100,17004.3,353.7,59384.3,3125.9,11.5,99,17144.8,140.4,62255.5,114.1,11.6,100,17313.9,169.1,62168.7,31.9,11.5,100,17534.3,220.4,61444.3,535.3,12.1,100,17686.2,151.9,61703.7,129.6,6.5,100,17885.4,199.2,61429.4,211.2,0.0,100,18200.4,314.9,61325.4,0.2,0.0,100,18440.1,239.8,61085.9,0.0,0.0,100,18733.9,293.8,58877.5,1914.6,0.0,99
2,MARABÁ (PA),PA,-5.52503,-49.94418,15161,36.2,86.0,5818.9,NaN,9218.2,0.0,1.7,100,6187.5,368.6,8849.6,0.0,1.7,100,6442.8,255.3,8594.3,0.0,1.7,100,6907.8,465.0,8129.3,0.0,1.7,100,7154.5,246.7,7882.6,0.0,1.7,100,7282.7,128.2,7617.7,136.7,1.7,100,7541.5,258.9,7380.2,115.4,1.7,100,7707.6,166.1,6680.3,649.2,1.7,100,8059.1,351.4,6977.3,0.0,2.4,100,8170.7,111.7,6720.5,145.9,1.7,99,8250.5,79.8,6751.7,31.4,5.2,99,8315.4,64.8,6636.3,81.6,5.5,100,8368.2,52.8,6643.6,21.8,5.2,100,8448.9,80.7,6578.9,5.6,5.4,100,8487.4,38.5,6101.2,448.5,1.7,99,8533.5,46.0,5913.0,592.3,0.0,100,8633.2,99.7,6404.9

In [0]:
# Actualizamos los datos antiguos con los nuevos
for index, row in df_new.iterrows():
    inc = 'Incremento deforestacion ' + str(row['year'])
    if row['year']==2019:
        df.loc[df['Municipio']==row['municipality'], 'Deforestacion ' + str(row['year'])] = df.loc[df['Municipio']==row['municipality'], 'Deforestacion ' + str(row['year']-1)] + row['areakm']
        df.loc[df['Municipio']==row['municipality'], inc] = row['areakm']
        df.loc[df['Municipio']==row['municipality'], 'Bosque ' + str(row['year'])] = df.loc[df['Municipio']==row['municipality'], 'Bosque ' + str(row['year']-1)] - row['areakm']
        df.loc[df['Municipio']==row['municipality'], 'Nubes ' + str(row['year'])] = 0
        df.loc[df['Municipio']==row['municipality'], 'No observado ' + str(row['year'])] = 0
        df.loc[df['Municipio']==row['municipality'], 'Check ' + str(row['year'])] = 100
    else:
        df.loc[df['Municipio']==row['municipality'], inc] = row['areakm']

In [0]:
df.loc[df['Municipio']=='PARAGOMINAS (PA)',:]

,Municipio,Estado,Latitud,Longitud,Area total,No bosque,Hidrografia,Deforestacion 2000,Incremento deforestacion 2000,Bosque 2000,Nubes 2000,No observado 2000,Check 2000,Deforestacion 2001,Incremento deforestacion 2001,Bosque 2001,Nubes 2001,No observado 2001,Check 2001,Deforestacion 2002,Incremento deforestacion 2002,Bosque 2002,Nubes 2002,No observado 2002,Check 2002,Deforestacion 2003,Incremento deforestacion 2003,Bosque 2003,Nubes 2003,No observado 2003,Check 2003,Deforestacion 2004,Incremento deforestacion 2004,Bosque 2004,Nubes 2004,No observado 2004,Check 2004,Deforestacion 2005,Incremento deforestacion 2005,Bosque 2005,Nubes 2005,No observado 2005,Check 2005,Deforestacion 2006,Incremento deforestacion 2006,Bosque 2006,Nubes 2006,No observado 2006,Check 2006,Deforestacion 2007,Incremento deforestacion 2007,Bosque 2007,Nubes 2007,No observado 2007,Check 2007,Deforestacion 2008,Incremento deforestacion 2008,Bosque 2008,Nubes 2008,No observado 2008,Check 2008,Deforestacion 2009,Incremento deforestacion 2009,Bosque 2009,Nubes 2009,No observado 2009,Check 2009,Deforestacion 2010,Incremento deforestacion 2010,Bosque 2010,Nubes 2010,No observado 2010,Check 2010,Deforestacion 2011,Incremento deforestacion 2011,Bosque 2011,Nubes 2011,No observado 2011,Check 2011,Deforestacion 2012,Incremento deforestacion 2012,Bosque 2012,Nubes 2012,No observado 2012,Check 2012,Deforestacion 2013,Incremento deforestacion 2013,Bosque 2013,Nubes 2013,No observado 2013,Check 2013,Deforestacion 2014,Incremento deforestacion 2014,Bosque 2014,Nubes 2014,No observado 2014,Check 2014,Deforestacion 2015,Incremento deforestacion 2015,Bosque 2015,Nubes 2015,No observado 2015,Check 2015,Deforestacion 2016,Incremento deforestacion 2016,Bosque 2016,Nubes 2016,No observado 2016,Check 2016,Deforestacion 2017,Incremento deforestacion 2017,Bosque 2017,Nubes 2017,No observado 2017,Check 2017,Deforestacion 2018,Incremento deforestacion 2018,Bosque 2018,Nubes 2018,No observado 2018,Check 2018,Deforestacion 2019,Incremento deforestacion 2019,Bosque 2019,Nubes 2019,No observado 2019,Check 2019
0,PARAGOMINAS (PA),PA,-3.12356,-47.40354,19465,7.1,36.0,7212.3,NaN,12208.2,0.0,1.4,100,7322.3,110.0,12098.2,0.0,1.4,100,7473.1,150.8,11947.0,0.4,1.4,100,7567.3,94.2,11853.2,0.0,1.4,100,7832.6,265.3,11587.9,0.0,1.4,100,8166.1,333.5,11253.9,0.5,1.4,100,8257.6,91.5,8732.4,2430.5,1.4,100,8349.1,91.5,10645.2,422.6,5.0,100,8415.4,63.339999,9094.9,1904.5,7.1,99,8536.7,120.407843,10703.6,121.8,59.8,100,8605.0,67.920908,10036.8,776.0,4.1,100,8641.0,35.098496,10301.9,474.8,4.2,100,8658.7,17.080393,7272.8,3486.4,4.0,99,8689.5,30.552451,7623.6,3105.0,3.8,100,8709.5,19.701243,7531.4,3179.6,1.4,99,8733.7,23.591894,9587.8,1100.4,0.0,100,8753.5,19.856212,9640.6,1027.8,0.0,100,8773.3,19.358134,10630.7,17.9,0.0,99,8791.7,18.048503,5999.4,4630.8,0.0,100,8817.470091,25.770091,5973.629909,0.0,0.0,100.0


In [0]:
print("El dataset viene ordenado por deforestacion, de mayor a menor. Para entrenar el modelo alteramos el orden de forma aleatoria.")
df = df.sample(frac=1)#.reset_index(drop=True)
df.reset_index(inplace=True, drop=True)
print(df.head())

In [0]:
print("El area total cubierta por los municipios es: " + str(df.sum(axis = 0, skipna = True)[4]) + " km^2, mientras el area total de la Amazonia es de 5,5M km^2.")
print("Por ello, podemos afirmar que estamos cubriendo una superficie que supone el 92% del total.")

print("Dibujamos el mapa de " + str(len(df['Latitud'])) + " municipios")
gmap = gmplot.GoogleMapPlotter(df['Latitud'][0], df['Longitud'][0], 5) # coordenadas del primer municipio del dataframe
gmap.heatmap(df['Latitud'], df['Longitud']) 
gmap.draw("mapa.html")
print("Mapa de calor generado")

In [0]:
def generate_fake_data(df, n_replic):
    fake_data=[]
    std=df.std()
    print(std)
    for idx, row in df.iterrows():
        for i in range(n_replic):
            munic = row['Municipio']+' fake_'+str(i)
            estado = row['Estado']
            lat = row['Latitud']+std['Latitud']*0.1*np.random.uniform(-1,1) # variacion del 10% de la desviación típica
            long = row['Longitud']+std['Longitud']*0.1*np.random.uniform(-1,1) # variacion del 10% de la desviación típica
            area = row['Area total']+row['Area total']*0.5*np.random.uniform(-1,1) # variacion del 50% del valor real
            no_bosq = row['No bosque']+row['No bosque']*0.5*np.random.uniform(-1,1) # variacion del 10% del valor real
            hidro = row['Hidrografia']+row['Hidrografia']*0.5*np.random.uniform(-1,1) # variacion del 10% del valor real
            temp = []
            for j in range(19):
                for k in range(n_vars_temp-2):
                    if k == 1: # incremento de deforestacion calculado a mano
                        if len(temp) > 6:
                            t = temp[-1] - temp[-7]
                            temp.append(t)
                            continue
                    proporcion = row[n_vars_temp*j+7+k]/row['Area total']
                    t = area*proporcion
                    t = t+t*0.1*np.random.uniform(-1,1)
                    temp.append(t)
                a = temp[-4] + sum(temp[-2:]) + no_bosq + hidro
                dif = area - a
                if dif < 0:
                    area += -dif
                    dif=0
                a = a + dif
                temp.append(dif)
                check = a/area*100
                temp.append(check)
            fixed = [munic, estado, lat, long, area, no_bosq, hidro]     
            fake_row = np.concatenate((fixed, temp))
            #for a, b in zip(row.values, fake_row):
            #    print(a,b)
            #print(len(fake_row))
            fake_data.append(fake_row)
    fake_df = pd.DataFrame(fake_data, columns=df.columns)
    return fake_df

fake_df = generate_fake_data(df, 10)
df_complete = df.append(fake_df)
#df_complete = df_complete.sort_values(by=['Municipio'])
df_complete.loc[:,'Latitud':] = df_complete.loc[:,'Latitud':].astype('float')
df_complete

In [0]:
fixed_vars = df.loc[:,['Municipio', 'Estado', 'Latitud', 'Longitud', 'Area total', 'Hidrografia', 'No bosque']]
temporal_vars = df.loc[:,'Deforestacion 2000':]
n_vars_temp = temporal_vars.shape[1]//19 # number of variables per year

In [0]:
fixed_vars = df_complete.loc[:,['Municipio', 'Estado', 'Latitud', 'Longitud', 'Area total', 'Hidrografia', 'No bosque']]
temporal_vars = df_complete.loc[:,'Deforestacion 2000':]
n_vars_temp = temporal_vars.shape[1]//19 # number of variables per year

In [0]:
df.head()

In [0]:
def series_to_supervised(temporal_vars, year_ini_train=0, year_ini_test=1, window_size=18, n_vars_temp=6):
    # ensure all data is float
    temporal_vars = temporal_vars.astype('float32')
    temporal_train = temporal_vars.iloc[:, n_vars_temp*year_ini_train:n_vars_temp*(year_ini_train+window_size)]
    temporal_test = temporal_vars.iloc[0:760, n_vars_temp*year_ini_test:] #testeamos con municipios reales
    return temporal_train, temporal_test

window_size=18
temporal_train, temporal_test = series_to_supervised(temporal_vars, 0, 1, window_size, n_vars_temp)

In [0]:
print(temporal_train.shape, temporal_test.shape)
print(temporal_train.head())

In [0]:
def preprocess_train(temporal_train, fixed_vars, n_vars_temp):
    # data structure for LSTM
    # normalize features
    x = temporal_train.iloc[:,0:-n_vars_temp].values
    y = temporal_train.iloc[:,-n_vars_temp].values.reshape(-1, 1) # reshape in 2D, we get 'deforestacion' as y
    scaler = MinMaxScaler(feature_range=(0, 1))
    x = scaler.fit_transform(x)
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    y = scaler2.fit_transform(y)
    print(temporal_train.shape, x.shape, y.shape)
    # structure in arrays to be the input of the LSTM
    vars_lstm=[]
    municipio_len=len(x[0])
    for mun in x:
        municipio=[]
        j=0
        while j < municipio_len:
            m=mun[j:j+n_vars_temp]
            if j==0:
                m[1]=-1		# marking missing values: Incremento deforestacion 1999/2000
            #elif m[5]!=municipio[0][5] or m[6]!=municipio[0][6]: # Hidrografia y No bosque son variables fijas!!!
            #	print("Hidrografia y No bosque varian de año en año")
            municipio.append(mun[j:j+n_vars_temp])
            j+=n_vars_temp
        vars_lstm.append(municipio)

    vars_lstm=np.array(vars_lstm)
    X1 = vars_lstm

    # data structure for Dense
    # One hot encoding 
    fixed_vars = pd.concat([fixed_vars,pd.get_dummies(fixed_vars['Estado'])],axis=1)
    # Drop column as it is now encoded
    fixed_vars = fixed_vars.drop('Estado',axis = 1)
    print(fixed_vars)
    scaler3 = MinMaxScaler(feature_range=(0, 1))
    x = scaler3.fit_transform(fixed_vars.iloc[:,1:]) # nombre del municipio en la primera columna
    X2 = np.array(x)
    return X1, X2, y, scaler, scaler2, scaler3
    
X1, X2, y, scaler, scaler2, scaler3 = preprocess_train(temporal_train, fixed_vars, n_vars_temp)

In [0]:
print("Ejemplo de input del modelo:")
print(len(X1), len(X2), len(y))

In [0]:
def create_model(mask=1):
    # LSTM para variables temporales
    input_temporal = Input(shape=(window_size-1, n_vars_temp))
    if mask==0:
        lstm = LSTM(5, activation='relu')(input_temporal) #, unroll =True) -> check what is is
    else:
        masking = Masking(mask_value=-1)(input_temporal)
        lstm = LSTM(5, activation='relu')(masking)
        #lstm_1 = LSTM(5, activation='relu', return_sequences = True)(masking)
        #lstm_2 = LSTM(5, activation='relu')(lstm_1)
    drop = Dropout(0.2)(lstm)
    dense_1 = Dense(16, activation = 'relu')(drop)
    dense_2 = Dense(16, activation = 'relu')(dense_1)
    
    # Dense para variables fijas
    input_fijo = Input(shape=(X2.shape[1],))
    dense_3 = Dense(32, activation = 'relu')(input_fijo)
    dense_4 = Dense(16, activation = 'relu')(dense_3)
    
    # concateno las 2 redes
    merge = concatenate([dense_2, dense_4])
    dense_5 = Dense(32, activation = 'relu')(merge)
    dense_6 = Dense(16, activation = 'relu')(dense_5)
    output = Dense(1, activation = 'relu')(dense_6)
    model = Model(inputs=[input_temporal, input_fijo], outputs=output)
    
    # summarize layers
    print(model.summary())
    # plot graph
    #plot_model(model, to_file='multiple_inputs.png')
    return model

model = create_model()
    
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])
# fit model
train_size = int(len(X1)*0.8)
model.fit([X1[0:train_size], X2[0:train_size]], y[0:train_size], epochs=10, batch_size=1, verbose=2, validation_data=([X1[train_size:], X2[train_size:]], y[train_size:]))


In [0]:
def preprocess_test(temporal_test, n_vars_temp, scaler, scaler2, scaler3):
    # data structure for LSTM
    # normalize features
    x = temporal_test.values[:,0:-n_vars_temp]
    y = temporal_test.values[:,-n_vars_temp].reshape(-1, 1) # reshape in 2D
    x = scaler.fit_transform(x)
    y = scaler2.fit_transform(y)
    print(temporal_test.shape, x.shape, y.shape)
    # structure in arrays to be the input of the LSTM
    vars_lstm=[]
    municipio_len=len(x[0])
    for mun in x:
        municipio=[]
        j=0
        while j < municipio_len:
            m=mun[j:j+n_vars_temp]
            if j==0:
                m[1]=-1		# marking missing values: Incremento deforestacion 1999/2000
            #elif m[5]!=municipio[0][5] or m[6]!=municipio[0][6]: # Hidrografia y No bosque son variables fijas!!!
            #	print("Hidrografia y No bosque varian de año en año")
            municipio.append(mun[j:j+n_vars_temp])
            j+=n_vars_temp
        vars_lstm.append(municipio)

    vars_lstm=np.array(vars_lstm)
    X1 = vars_lstm
    return X1, y

X1_test, y_test = preprocess_test(temporal_test, n_vars_temp, scaler, scaler2, scaler3)
X2 = X2[0:760]

In [0]:
print(len(X1_test[0]), len(X2[0]), len(y_test[0]))

In [0]:
# evaluate model on new data
y_pred = model.predict([X1_test, X2])

In [0]:
#for i in range(len(y_pred)):
	#print('Expected', y[i], 'Predicted', y_pred[i])

In [0]:
# rescaling
y=scaler2.inverse_transform(y)
y_pred=scaler2.inverse_transform(y_pred)

In [0]:
errores = []
abajo = 0
num_test = len(y_pred)
for i in range(num_test):
    delta = y[i] - y_pred[i]
    if delta > 0:
        abajo += 1
    error = abs(delta)
    print('Expected', y[i], 'Predicted', y_pred[i], 'Error', str(error))
    errores.append(error)
    if error > 1000:
        print (error)
print('Error absoluto medio', np.array(errores).mean(), "con una muestra de", num_test, "municipios")
print('La predicción es menor que el dato real',abajo, "veces")